# 4B Illustrative economy Ib: smooth ambiguity


This notebook is to supplement discussion in [Section 4](sec4_IllustrativeEconI.ipynb) by focusing on a different aspect -- the impact of smooth ambiguity.

The same with [Section 4](sec4_IllustrativeEconI.ipynb), the penalty paramters are $\xi_a$ and $\xi_r$. 

Without specifically pointed out, $\xi_r = 1$ in this example. And the smooth ambiguity parameter $\xi_a$ values we experiment with are $\{0.02, 0.01, 0.005, 0.0025\}$.

## 4B.1 Post-jump continuation value functions


Conditioned on each of the damage functions, $m = 1, 2, \dots, 20$. Solve for the corresponding $\phi_m(y)$:


$$
\begin{align*}
0 = \max_{\tilde e}  \min_h \min_{\omega_j, \sum_{\ell =1}^L \omega_\ell  = 1}
& - \delta \phi_m(y)    +  \eta \log \tilde e    \cr
& + \frac {d \phi_m(y)}{d y} {\tilde e}  \varsigma \cdot h  + {\frac {(\eta - 1)} \delta }\left[\gamma_1 +  \gamma_2 y + \gamma_3^m (y- {\overline y} ) \right] {\tilde e} \varsigma \cdot h + {\frac {\xi_r} 2} h'h \cr 
& + \frac {d \phi_m(y)}{d y}  \sum_{\ell=1}^L \omega_\ell  \theta_\ell {\tilde e} + {\frac 1 2} \frac {d^2 \phi_m(y)}{(dy)^2} |\varsigma|^2 \tilde e^2  \cr
&+ {\frac {(\eta - 1)} \delta}  \left( \left[ \gamma_1 + \gamma_2 y + \gamma_3^m (y - \overline y) \right]   \sum_{\ell=1}^L \omega_\ell \theta_\ell {\tilde e} + {\frac 1 2} (\gamma_2 + 
\gamma_3^m) |\varsigma|^2 \tilde e^2 \right) \cr
&+ \xi_a \sum_{\ell = 1}^L \omega_\ell \left( \log \omega_\ell - \log \pi_\ell \right).
\end{align*}
$$

In [ ]:
import numpy as np
import pandas as pd
from src.model import solve_hjb_y, solve_hjb_y_jump
from src.utilities import find_nearest_value, solve_post_jump
from src.simulation import simulate_jump, no_jump_simulation
import pickle

In [ ]:
# Preference
η = 0.032
δ = 0.01

# Climate sensitivity
θ_list = pd.read_csv('data/model144.csv', header=None).to_numpy()[:, 0] / 1000.
πa_o = np.ones_like(θ_list) / len(θ_list)

# Damage functions
σ_y = 1.2 * np.mean(θ_list)
y_underline = 1.5
y_bar = 2.
γ_1 = 1.7675 / 10000
γ_2 = 0.0022 * 2
γ_3 = np.linspace(0., 1. / 3, 20)
πd_o = np.ones_like(γ_3) / len(γ_3)

# capital evolution
α = 0.115
i_over_k = 0.09
K0 = 85 / α

# state variable
y_step = .01
y_grid_long = np.arange(0., 5., y_step)
y_grid_short = np.arange(0., 2.1 + y_step, y_step)
n_bar = find_nearest_value(y_grid_long, y_bar) 

In [ ]:
# # Prepare ϕ conditional on low, high, extreme damage
# ξ_r = 1.
# v_list = {}
# e_tilde_list = {}
# for ξ_a_i in [0.02, 0.01, 0.005, 0.0025]:
#     model_args_list = []
#     for γ_3_m in γ_3:
#         model_arg = (η, δ, σ_y, y_bar, γ_1, γ_2, γ_3_m, θ_list, πa_o, ξ_r, ξ_a_i)
#         model_args_list.append((y_grid_long, model_arg, None, 1., 1e-8, 5_000, False))
#     v_list[ξ_a_i], e_tilde_list[ξ_a_i] = solve_post_jump(y_grid_long, γ_3, solve_hjb_y, model_args_list)

In [ ]:
# pickle.dump(v_list, open("v_list_sa", "wb"))
# pickle.dump(e_tilde_list, open("e_tilde_list_sa", "wb"))
v_list = pickle.load(open("v_list_sa", "rb"))
e_tilde_list = pickle.load(open("e_tilde_list_sa", "rb"))

## 4B Pre-jump value function

The pre-jump value function has a similar structure with two exceptions:
  -  we include the intensity function discussed earlier and 
  -  we introduce robustness concerns for both the 
intensity and distribution over the alternative $\gamma_3^m$ coefficients.  

Given these modifications, we include:


$$
\mathcal J (y) \sum_{m=1}^M g_m \pi_m \left[ \phi_m(\overline y) - \phi(y) \right]
+ \xi_r {\mathcal J}(y)  \sum_{m=1}^M \pi_m \left( 1 - g_m + g_m \log g_m \right)\pi_m 
$$

in the HJB and solve for pre-jump value function $\phi(y)$ on $[0, \overline{y}]$:

$$
\begin{align*}
0 = \max_{\tilde e}  \min_h \min_{\omega_j, \sum_{\ell =1}^L \omega_\ell  = 1} \min_{g_m \geqslant 0}
& - \delta \phi(y)    +  \eta \log \tilde e    \cr
& + \frac {d \phi(y)}{d y} {\tilde e}  \varsigma \cdot h  + {\frac {(\eta - 1)} \delta }\left[\gamma_1 +  \gamma_2 y) \right] {\tilde e} \varsigma \cdot h + {\frac {\xi_r} 2} h'h \cr 
& + \frac {d \phi(y)}{d y}  \sum_{\ell=1}^L \omega_\ell  \theta_\ell {\tilde e} + {\frac 1 2} \frac {d^2 \phi(y)}{(dy)^2} |\varsigma|^2 \tilde e^2  \cr
&+ {\frac {(\eta - 1)} \delta}  \left( \left[ \gamma_1 + \gamma_2 y\right]   \sum_{\ell=1}^L \omega_\ell \theta_\ell {\tilde e} + {\frac 1 2} \gamma_2  |\varsigma|^2 \tilde e^2 \right) \cr
&+ \xi_a \sum_{\ell = 1}^L \omega_\ell \left( \log \omega_\ell - \log \pi_\ell \right)\cr
&+ \mathcal J (y) \sum_{m=1}^M g_m \pi_m \left[ \phi_m(\overline y) - \phi(y) \right]
+ \xi_r {\mathcal J}(y)  \sum_{m=1}^M \pi_m \left( 1 - g_m + g_m \log g_m \right)\pi_m 
\end{align*}
$$

In [ ]:
ξ_a_list = [0.02, 0.01, 0.005, 0.0025]

In [ ]:
# pre_jump_res = {}
# for ξ_a_i in ξ_a_list:
#     ϕ_list = v_list[ξ_a_i]
#     certainty_equivalent = -ξ_r * np.log(
#         np.average(
#             np.exp(-1. / ξ_r * np.array(ϕ_list)), axis=0, weights=πd_o))
#     # Change grid from 0-4 to 0-2
#     ϕ_i = np.array(
#         [temp[n_bar] * np.ones_like(y_grid_short) for temp in ϕ_list])

#     # Compute ϕ with jump (impose boundary condition)
#     model_args = (η, δ, σ_y, y_underline, y_bar, γ_1, γ_2, γ_3, θ_list, πa_o, ϕ_i, πd_o,
#                   ξ_r, ξ_r, ξ_a_i)
#     model_res = solve_hjb_y_jump(y_grid_short,
#                                  model_args,
#                                  v0=None,
#                                  ϵ=1.,
#                                  tol=1e-8,
#                                  max_iter=5_000,
#                                  print_iteration=False)
#     simulation_res = no_jump_simulation(model_res, dt=1/4)
#     pre_jump_res[ξ_a_i] = dict(model_res=model_res,
#                            simulation_res=simulation_res,
#                            certainty_equivalent=certainty_equivalent)

In [ ]:
# pickle.dump(pre_jump_res, open("pre_jump_res_sa", "wb"))
pre_jump_res = pickle.load(open("pre_jump_res_sa", "rb"))

## 4B.3 Emission and $\log SCC$ as a function of temperature anomaly

In [ ]:
from src.plots import plot_ems_app
plot_ems_app(pre_jump_res, y_grid_short, ξ_a_list)

In [ ]:
from src.plots import plot_logscc
args_scc = (α, η, i_over_k, K0, γ_1, γ_2)
plot_logscc(pre_jump_res, y_grid_short, 1.5, ξ_a_list, args_scc).write_html('plot_logscc.html')
IFrame(src ='plot_logscc.html', width=1090, height=500)

## 4B.4 Emission and logarithm of social cost of carbon trajectories

The figure shows emission and $\log SCC$ over time before temperature anomaly reaches the upper bound of jump threshold, 2$^o C$.


In [ ]:
emission_traj = plot_ems_app(pre_jump_res, y_grid_short, ξ_a_list, dt=1/4, model_res=False)
emission_traj.update_yaxes(range=[0,14])